In [1]:
import $ivy.`org.apache.spark::spark-sql:3.5.0`

import $ivy.$

In [2]:
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import org.apache.log4j.{Level, Logger}

In [3]:
import org.apache.spark.sql._
import org.apache.spark.sql.functions._

import org.apache.spark.sql._
import org.apache.spark.sql.functions._

In [4]:
val spark = SparkSession
                .builder()
                .master("local[*]")
                .appName("Dataframe API")
                .getOrCreate()

import spark.implicits._

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
24/09/06 14:53:01 INFO SparkContext: Running Spark version 3.5.0
24/09/06 14:53:01 INFO SparkContext: OS info Linux, 6.8.0-40-generic, amd64
24/09/06 14:53:01 INFO SparkContext: Java version 11.0.24
24/09/06 14:53:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/09/06 14:53:01 INFO ResourceUtils: ==============================================================
24/09/06 14:53:01 INFO ResourceUtils: No custom resources configured for spark.driver.
24/09/06 14:53:01 INFO ResourceUtils: ==============================================================
24/09/06 14:53:01 INFO SparkContext: Submitted application: Dataframe API
24/09/06 14:53:01 INFO ResourceProfile: Default ResourceProfile created, executor resources: Map(cores -> name: cores, amount: 1, script: , vendor: , memory -> name: memory, amount: 1024, script: , vendor: , offHea

spark: SparkSession = org.apache.spark.sql.SparkSession@11c4a98b
import spark.implicits._

In [5]:
println(s"spark.version == ${spark.version}")

spark.version == 3.5.0


In [6]:
val customerDF = spark.read.format("json").load("data/customer_data.json")
customerDF.show(5, false)

24/09/06 14:53:03 INFO SharedState: Setting hive.metastore.warehouse.dir ('null') to the value of spark.sql.warehouse.dir.
24/09/06 14:53:03 INFO SharedState: Warehouse path is 'file:/home/vadim/workspace/Spark/SQL/spark-warehouse'.
24/09/06 14:53:04 INFO InMemoryFileIndex: It took 26 ms to list leaf files for 1 paths.
24/09/06 14:53:04 INFO InMemoryFileIndex: It took 1 ms to list leaf files for 1 paths.
24/09/06 14:53:05 INFO FileSourceStrategy: Pushed Filters: 
24/09/06 14:53:05 INFO FileSourceStrategy: Post-Scan Filters: 
24/09/06 14:53:05 INFO MemoryStore: Block broadcast_0 stored as values in memory (estimated size 198.5 KiB, free 4.5 GiB)
24/09/06 14:53:06 INFO MemoryStore: Block broadcast_0_piece0 stored as bytes in memory (estimated size 34.0 KiB, free 4.5 GiB)
24/09/06 14:53:06 INFO BlockManagerInfo: Added broadcast_0_piece0 in memory on ubuntu:34403 (size: 34.0 KiB, free: 4.5 GiB)
24/09/06 14:53:06 INFO SparkContext: Created broadcast 0 from load at cell6.sc:1
24/09/06 14:53:

+------------------------------------------------------+-------------------+--------------+----------+------------------------+---------------+----------------+
|Address                                               |Birthdate          |Country       |CustomerID|Email                   |Name           |Username        |
+------------------------------------------------------+-------------------+--------------+----------+------------------------+---------------+----------------+
|Unit 1047 Box 4089\nDPO AA 57348                      |1994-02-20 00:46:27|United Kingdom|12346     |cooperalexis@hotmail.com|Lindsay Cowan  |valenciajennifer|
|55711 Janet Plaza Apt. 865\nChristinachester, CT 62716|1988-06-21 00:15:34|Iceland       |12347     |timothy78@hotmail.com   |Katherine David|hillrachel      |
|Unit 2676 Box 9352\nDPO AA 38560                      |1974-11-26 15:30:20|Finland       |12348     |tcrawford@gmail.com     |Leslie Martinez|serranobrian    |
|2765 Powers Meadow\nHeatherfurt, 

customerDF: DataFrame = [Address: string, Birthdate: string ... 5 more fields]

In [7]:
customerDF.createOrReplaceTempView("customer")

24/09/06 14:53:07 INFO BlockManagerInfo: Removed broadcast_0_piece0 on ubuntu:34403 in memory (size: 34.0 KiB, free: 4.5 GiB)
24/09/06 14:53:07 INFO BlockManagerInfo: Removed broadcast_1_piece0 on ubuntu:34403 in memory (size: 7.4 KiB, free: 4.5 GiB)
24/09/06 14:53:07 INFO BlockManagerInfo: Removed broadcast_3_piece0 on ubuntu:34403 in memory (size: 7.6 KiB, free: 4.5 GiB)


In [8]:
spark.sql("select Birthdate, Country from customer")

res8: DataFrame = [Birthdate: string, Country: string]

In [9]:
spark.sql("select Birthdate, Country from customer").show(5, false)

24/09/06 14:53:08 INFO FileSourceStrategy: Pushed Filters: 
24/09/06 14:53:08 INFO FileSourceStrategy: Post-Scan Filters: 
24/09/06 14:53:08 INFO CodeGenerator: Code generated in 7.827997 ms
24/09/06 14:53:08 INFO MemoryStore: Block broadcast_4 stored as values in memory (estimated size 198.4 KiB, free 4.5 GiB)
24/09/06 14:53:08 INFO MemoryStore: Block broadcast_4_piece0 stored as bytes in memory (estimated size 34.0 KiB, free 4.5 GiB)
24/09/06 14:53:08 INFO BlockManagerInfo: Added broadcast_4_piece0 in memory on ubuntu:34403 (size: 34.0 KiB, free: 4.5 GiB)
24/09/06 14:53:08 INFO SparkContext: Created broadcast 4 from show at cell9.sc:1
24/09/06 14:53:08 INFO FileSourceScanExec: Planning scan with bin packing, max size: 4194304 bytes, open cost is considered as scanning 4194304 bytes.
24/09/06 14:53:08 INFO SparkContext: Starting job: show at cell9.sc:1
24/09/06 14:53:08 INFO DAGScheduler: Got job 2 (show at cell9.sc:1) with 1 output partitions
24/09/06 14:53:08 INFO DAGScheduler: Fina

+-------------------+--------------+
|Birthdate          |Country       |
+-------------------+--------------+
|1994-02-20 00:46:27|United Kingdom|
|1988-06-21 00:15:34|Iceland       |
|1974-11-26 15:30:20|Finland       |
|1977-05-06 23:57:35|Italy         |
|1996-09-13 19:14:27|Norway        |
+-------------------+--------------+
only showing top 5 rows



In [10]:
spark.sql("select Birthdate as Date, Country from customer").show(5, false)

24/09/06 14:53:09 INFO FileSourceStrategy: Pushed Filters: 
24/09/06 14:53:09 INFO FileSourceStrategy: Post-Scan Filters: 
24/09/06 14:53:09 INFO MemoryStore: Block broadcast_6 stored as values in memory (estimated size 198.4 KiB, free 4.5 GiB)
24/09/06 14:53:09 INFO MemoryStore: Block broadcast_6_piece0 stored as bytes in memory (estimated size 34.0 KiB, free 4.5 GiB)
24/09/06 14:53:09 INFO BlockManagerInfo: Added broadcast_6_piece0 in memory on ubuntu:34403 (size: 34.0 KiB, free: 4.5 GiB)
24/09/06 14:53:09 INFO SparkContext: Created broadcast 6 from show at cell10.sc:1
24/09/06 14:53:09 INFO FileSourceScanExec: Planning scan with bin packing, max size: 4194304 bytes, open cost is considered as scanning 4194304 bytes.
24/09/06 14:53:09 INFO SparkContext: Starting job: show at cell10.sc:1
24/09/06 14:53:09 INFO DAGScheduler: Got job 3 (show at cell10.sc:1) with 1 output partitions
24/09/06 14:53:09 INFO DAGScheduler: Final stage: ResultStage 3 (show at cell10.sc:1)
24/09/06 14:53:09 IN

+-------------------+--------------+
|Date               |Country       |
+-------------------+--------------+
|1994-02-20 00:46:27|United Kingdom|
|1988-06-21 00:15:34|Iceland       |
|1974-11-26 15:30:20|Finland       |
|1977-05-06 23:57:35|Italy         |
|1996-09-13 19:14:27|Norway        |
+-------------------+--------------+
only showing top 5 rows



In [11]:
spark.sql("select Birthdate as Date, Country, true as Flag from customer").show(5, false)

24/09/06 14:53:09 INFO FileSourceStrategy: Pushed Filters: 
24/09/06 14:53:09 INFO FileSourceStrategy: Post-Scan Filters: 
24/09/06 14:53:09 INFO CodeGenerator: Code generated in 11.202559 ms
24/09/06 14:53:09 INFO MemoryStore: Block broadcast_8 stored as values in memory (estimated size 198.4 KiB, free 4.5 GiB)
24/09/06 14:53:09 INFO MemoryStore: Block broadcast_8_piece0 stored as bytes in memory (estimated size 34.0 KiB, free 4.5 GiB)
24/09/06 14:53:09 INFO BlockManagerInfo: Added broadcast_8_piece0 in memory on ubuntu:34403 (size: 34.0 KiB, free: 4.5 GiB)
24/09/06 14:53:09 INFO SparkContext: Created broadcast 8 from show at cell11.sc:1
24/09/06 14:53:09 INFO FileSourceScanExec: Planning scan with bin packing, max size: 4194304 bytes, open cost is considered as scanning 4194304 bytes.
24/09/06 14:53:09 INFO SparkContext: Starting job: show at cell11.sc:1
24/09/06 14:53:09 INFO DAGScheduler: Got job 4 (show at cell11.sc:1) with 1 output partitions
24/09/06 14:53:09 INFO DAGScheduler: 

+-------------------+--------------+----+
|Date               |Country       |Flag|
+-------------------+--------------+----+
|1994-02-20 00:46:27|United Kingdom|true|
|1988-06-21 00:15:34|Iceland       |true|
|1974-11-26 15:30:20|Finland       |true|
|1977-05-06 23:57:35|Italy         |true|
|1996-09-13 19:14:27|Norway        |true|
+-------------------+--------------+----+
only showing top 5 rows



In [12]:
spark.sql("select Birthdate as Date, Country, true as Flag from customer").printSchema

root
 |-- Date: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Flag: boolean (nullable = false)



In [13]:
spark.sql("select * from customer where Country = 'Norway'").show(5, false)

24/09/06 14:53:10 INFO FileSourceStrategy: Pushed Filters: IsNotNull(Country),EqualTo(Country,Norway)
24/09/06 14:53:10 INFO FileSourceStrategy: Post-Scan Filters: isnotnull(Country#10),(Country#10 = Norway)
24/09/06 14:53:10 INFO CodeGenerator: Code generated in 15.21472 ms
24/09/06 14:53:10 INFO MemoryStore: Block broadcast_10 stored as values in memory (estimated size 198.4 KiB, free 4.5 GiB)
24/09/06 14:53:10 INFO MemoryStore: Block broadcast_10_piece0 stored as bytes in memory (estimated size 34.0 KiB, free 4.5 GiB)
24/09/06 14:53:10 INFO BlockManagerInfo: Added broadcast_10_piece0 in memory on ubuntu:34403 (size: 34.0 KiB, free: 4.5 GiB)
24/09/06 14:53:10 INFO SparkContext: Created broadcast 10 from show at cell13.sc:1
24/09/06 14:53:10 INFO FileSourceScanExec: Planning scan with bin packing, max size: 4194304 bytes, open cost is considered as scanning 4194304 bytes.
24/09/06 14:53:10 INFO SparkContext: Starting job: show at cell13.sc:1
24/09/06 14:53:10 INFO DAGScheduler: Got jo

+--------------------------------------------------+-------------------+-------+----------+-------------------------+--------------+---------------+
|Address                                           |Birthdate          |Country|CustomerID|Email                    |Name          |Username       |
+--------------------------------------------------+-------------------+-------+----------+-------------------------+--------------+---------------+
|17677 Mark Crest\nWalterberg, IA 39017            |1996-09-13 19:14:27|Norway |12350     |amyholland@yahoo.com     |Natalie Ford  |gregoryharrison|
|50047 Smith Point Suite 162\nWilkinsstad, PA 04106|1969-06-21 03:39:20|Norway |12352     |vcarter@hotmail.com      |Dana Clarke   |hmyers         |
|0297 Jacob Ranch Apt. 019\nNorth Judith, NV 27455 |1990-06-01 14:49:52|Norway |12381     |douglaschavez@hotmail.com|Matthew Jones |stephanie68    |
|3102 Hopkins Walk\nAndrechester, MD 54461         |1976-06-19 08:10:24|Norway |12430     |crystalromero@h

In [14]:
spark.sql("select * from customer where Country <> 'Iceland'").show(5, false)

24/09/06 14:53:11 INFO BlockManagerInfo: Removed broadcast_5_piece0 on ubuntu:34403 in memory (size: 7.1 KiB, free: 4.5 GiB)
24/09/06 14:53:11 INFO BlockManagerInfo: Removed broadcast_2_piece0 on ubuntu:34403 in memory (size: 34.0 KiB, free: 4.5 GiB)
24/09/06 14:53:11 INFO BlockManagerInfo: Removed broadcast_11_piece0 on ubuntu:34403 in memory (size: 7.9 KiB, free: 4.5 GiB)
24/09/06 14:53:11 INFO BlockManagerInfo: Removed broadcast_7_piece0 on ubuntu:34403 in memory (size: 7.1 KiB, free: 4.5 GiB)
24/09/06 14:53:11 INFO BlockManagerInfo: Removed broadcast_6_piece0 on ubuntu:34403 in memory (size: 34.0 KiB, free: 4.5 GiB)
24/09/06 14:53:11 INFO BlockManagerInfo: Removed broadcast_9_piece0 on ubuntu:34403 in memory (size: 7.2 KiB, free: 4.5 GiB)
24/09/06 14:53:11 INFO BlockManagerInfo: Removed broadcast_4_piece0 on ubuntu:34403 in memory (size: 34.0 KiB, free: 4.5 GiB)
24/09/06 14:53:11 INFO BlockManagerInfo: Removed broadcast_8_piece0 on ubuntu:34403 in memory (size: 34.0 KiB, free: 4.5 

+--------------------------------------------------+-------------------+--------------+----------+------------------------+---------------+----------------+
|Address                                           |Birthdate          |Country       |CustomerID|Email                   |Name           |Username        |
+--------------------------------------------------+-------------------+--------------+----------+------------------------+---------------+----------------+
|Unit 1047 Box 4089\nDPO AA 57348                  |1994-02-20 00:46:27|United Kingdom|12346     |cooperalexis@hotmail.com|Lindsay Cowan  |valenciajennifer|
|Unit 2676 Box 9352\nDPO AA 38560                  |1974-11-26 15:30:20|Finland       |12348     |tcrawford@gmail.com     |Leslie Martinez|serranobrian    |
|2765 Powers Meadow\nHeatherfurt, CT 53165         |1977-05-06 23:57:35|Italy         |12349     |dustin37@yahoo.com      |Brad Cardenas  |charleshudson   |
|17677 Mark Crest\nWalterberg, IA 39017            |1996-0

In [15]:
spark.sql("select * from customer order by CustomerID desc").show(5, false)

24/09/06 14:53:11 INFO FileSourceStrategy: Pushed Filters: 
24/09/06 14:53:11 INFO FileSourceStrategy: Post-Scan Filters: 
24/09/06 14:53:12 INFO CodeGenerator: Code generated in 12.383984 ms
24/09/06 14:53:12 INFO MemoryStore: Block broadcast_14 stored as values in memory (estimated size 198.4 KiB, free 4.5 GiB)
24/09/06 14:53:12 INFO MemoryStore: Block broadcast_14_piece0 stored as bytes in memory (estimated size 34.0 KiB, free 4.5 GiB)
24/09/06 14:53:12 INFO BlockManagerInfo: Added broadcast_14_piece0 in memory on ubuntu:34403 (size: 34.0 KiB, free: 4.5 GiB)
24/09/06 14:53:12 INFO SparkContext: Created broadcast 14 from show at cell15.sc:1
24/09/06 14:53:12 INFO FileSourceScanExec: Planning scan with bin packing, max size: 4194304 bytes, open cost is considered as scanning 4194304 bytes.
24/09/06 14:53:12 INFO SparkContext: Starting job: show at cell15.sc:1
24/09/06 14:53:12 INFO DAGScheduler: Got job 7 (show at cell15.sc:1) with 1 output partitions
24/09/06 14:53:12 INFO DAGSchedul

+----------------------------------------------------+-------------------+--------------+----------+-----------------------+--------------------+-----------------+
|Address                                             |Birthdate          |Country       |CustomerID|Email                  |Name                |Username         |
+----------------------------------------------------+-------------------+--------------+----------+-----------------------+--------------------+-----------------+
|6942 Connie Skyway\nPatrickville, WA 16551          |1973-10-24 00:52:10|United Kingdom|12989     |amber97@hotmail.com    |Brandon Contreras   |ecasey           |
|79375 David Neck\nWest Matthewton, NJ 92863         |1971-05-04 22:20:10|United Kingdom|12988     |erica98@gmail.com      |Gabriel Romero      |qknight          |
|00881 West Flat\nNorth Emily, IL 32130              |1997-03-05 19:20:57|United Kingdom|12987     |vkeith@yahoo.com       |Christopher Lawrence|smcintyre        |
|499 Jonathan St

In [16]:
spark.sql("select Birthdate, date_format(Birthdate, 'yyyy-MM-dd') as Date from customer").show(5, false)

24/09/06 14:53:12 INFO FileSourceStrategy: Pushed Filters: 
24/09/06 14:53:12 INFO FileSourceStrategy: Post-Scan Filters: 
24/09/06 14:53:12 INFO CodeGenerator: Code generated in 11.513564 ms
24/09/06 14:53:12 INFO MemoryStore: Block broadcast_16 stored as values in memory (estimated size 198.4 KiB, free 4.5 GiB)
24/09/06 14:53:12 INFO MemoryStore: Block broadcast_16_piece0 stored as bytes in memory (estimated size 34.0 KiB, free 4.5 GiB)
24/09/06 14:53:12 INFO BlockManagerInfo: Added broadcast_16_piece0 in memory on ubuntu:34403 (size: 34.0 KiB, free: 4.5 GiB)
24/09/06 14:53:12 INFO SparkContext: Created broadcast 16 from show at cell16.sc:1
24/09/06 14:53:12 INFO FileSourceScanExec: Planning scan with bin packing, max size: 4194304 bytes, open cost is considered as scanning 4194304 bytes.
24/09/06 14:53:12 INFO SparkContext: Starting job: show at cell16.sc:1
24/09/06 14:53:12 INFO DAGScheduler: Got job 8 (show at cell16.sc:1) with 1 output partitions
24/09/06 14:53:12 INFO DAGSchedul

+-------------------+----------+
|Birthdate          |Date      |
+-------------------+----------+
|1994-02-20 00:46:27|1994-02-20|
|1988-06-21 00:15:34|1988-06-21|
|1974-11-26 15:30:20|1974-11-26|
|1977-05-06 23:57:35|1977-05-06|
|1996-09-13 19:14:27|1996-09-13|
+-------------------+----------+
only showing top 5 rows



In [17]:
spark.sql("select Name, Username, array(Name, Username) as Identity from customer").show(5, false)

24/09/06 14:53:13 INFO FileSourceStrategy: Pushed Filters: 
24/09/06 14:53:13 INFO FileSourceStrategy: Post-Scan Filters: 
24/09/06 14:53:13 INFO CodeGenerator: Code generated in 7.427655 ms
24/09/06 14:53:13 INFO MemoryStore: Block broadcast_18 stored as values in memory (estimated size 198.4 KiB, free 4.5 GiB)
24/09/06 14:53:13 INFO MemoryStore: Block broadcast_18_piece0 stored as bytes in memory (estimated size 34.0 KiB, free 4.5 GiB)
24/09/06 14:53:13 INFO BlockManagerInfo: Added broadcast_18_piece0 in memory on ubuntu:34403 (size: 34.0 KiB, free: 4.5 GiB)
24/09/06 14:53:13 INFO SparkContext: Created broadcast 18 from show at cell17.sc:1
24/09/06 14:53:13 INFO FileSourceScanExec: Planning scan with bin packing, max size: 4194304 bytes, open cost is considered as scanning 4194304 bytes.
24/09/06 14:53:13 INFO SparkContext: Starting job: show at cell17.sc:1
24/09/06 14:53:13 INFO DAGScheduler: Got job 9 (show at cell17.sc:1) with 1 output partitions
24/09/06 14:53:13 INFO DAGSchedule

+---------------+----------------+---------------------------------+
|Name           |Username        |Identity                         |
+---------------+----------------+---------------------------------+
|Lindsay Cowan  |valenciajennifer|[Lindsay Cowan, valenciajennifer]|
|Katherine David|hillrachel      |[Katherine David, hillrachel]    |
|Leslie Martinez|serranobrian    |[Leslie Martinez, serranobrian]  |
|Brad Cardenas  |charleshudson   |[Brad Cardenas, charleshudson]   |
|Natalie Ford   |gregoryharrison |[Natalie Ford, gregoryharrison]  |
+---------------+----------------+---------------------------------+
only showing top 5 rows



In [18]:
spark.sql("select Country, count(*) as count from customer group by Country order by count desc").show(5, false)

24/09/06 14:53:13 INFO FileSourceStrategy: Pushed Filters: 
24/09/06 14:53:13 INFO FileSourceStrategy: Post-Scan Filters: 
24/09/06 14:53:13 INFO CodeGenerator: Code generated in 33.192484 ms
24/09/06 14:53:13 INFO MemoryStore: Block broadcast_20 stored as values in memory (estimated size 198.4 KiB, free 4.5 GiB)
24/09/06 14:53:13 INFO MemoryStore: Block broadcast_20_piece0 stored as bytes in memory (estimated size 34.0 KiB, free 4.5 GiB)
24/09/06 14:53:13 INFO BlockManagerInfo: Added broadcast_20_piece0 in memory on ubuntu:34403 (size: 34.0 KiB, free: 4.5 GiB)
24/09/06 14:53:13 INFO SparkContext: Created broadcast 20 from show at cell18.sc:1
24/09/06 14:53:13 INFO FileSourceScanExec: Planning scan with bin packing, max size: 4194304 bytes, open cost is considered as scanning 4194304 bytes.
24/09/06 14:53:13 INFO DAGScheduler: Registering RDD 43 (show at cell18.sc:1) as input to shuffle 0
24/09/06 14:53:13 INFO DAGScheduler: Got map stage job 10 (show at cell18.sc:1) with 1 output part

+--------------+-----+
|Country       |count|
+--------------+-----+
|United Kingdom|124  |
|Germany       |86   |
|France        |86   |
|Spain         |30   |
|Belgium       |25   |
+--------------+-----+
only showing top 5 rows



In [19]:
spark.sql("""
select
  Country
  , date_format(Birthdate, 'yyyy-MM-dd') as bd
  , min(CustomerID)
  , max(CustomerID)
from customer
group by Country, bd
order by Country""")
.show(5, false)

24/09/06 14:53:40 INFO FileSourceStrategy: Pushed Filters: 
24/09/06 14:53:40 INFO FileSourceStrategy: Post-Scan Filters: 
24/09/06 14:53:40 INFO CodeGenerator: Code generated in 10.732957 ms
24/09/06 14:53:40 INFO MemoryStore: Block broadcast_23 stored as values in memory (estimated size 198.4 KiB, free 4.5 GiB)
24/09/06 14:53:40 INFO MemoryStore: Block broadcast_23_piece0 stored as bytes in memory (estimated size 34.0 KiB, free 4.5 GiB)
24/09/06 14:53:40 INFO BlockManagerInfo: Added broadcast_23_piece0 in memory on ubuntu:34403 (size: 34.0 KiB, free: 4.5 GiB)
24/09/06 14:53:40 INFO SparkContext: Created broadcast 23 from show at cell19.sc:10
24/09/06 14:53:40 INFO FileSourceScanExec: Planning scan with bin packing, max size: 4194304 bytes, open cost is considered as scanning 4194304 bytes.
24/09/06 14:53:40 INFO BlockManagerInfo: Removed broadcast_22_piece0 on ubuntu:34403 in memory (size: 19.4 KiB, free: 4.5 GiB)
24/09/06 14:53:40 INFO DAGScheduler: Registering RDD 52 (show at cell1

+---------+----------+---------------+---------------+
|Country  |bd        |min(CustomerID)|max(CustomerID)|
+---------+----------+---------------+---------------+
|Australia|1966-09-17|12422          |12422          |
|Australia|1967-11-29|12424          |12424          |
|Australia|1985-12-30|12386          |12386          |
|Australia|1986-08-28|12434          |12434          |
|Australia|1987-02-26|12393          |12393          |
+---------+----------+---------------+---------------+
only showing top 5 rows



In [20]:
spark.sql("select count(*) from customer").show

24/09/06 14:53:45 INFO FileSourceStrategy: Pushed Filters: 
24/09/06 14:53:45 INFO FileSourceStrategy: Post-Scan Filters: 
24/09/06 14:53:45 INFO CodeGenerator: Code generated in 5.614474 ms
24/09/06 14:53:45 INFO MemoryStore: Block broadcast_26 stored as values in memory (estimated size 198.4 KiB, free 4.5 GiB)
24/09/06 14:53:45 INFO MemoryStore: Block broadcast_26_piece0 stored as bytes in memory (estimated size 34.0 KiB, free 4.5 GiB)
24/09/06 14:53:45 INFO BlockManagerInfo: Added broadcast_26_piece0 in memory on ubuntu:34403 (size: 34.0 KiB, free: 4.5 GiB)
24/09/06 14:53:45 INFO SparkContext: Created broadcast 26 from show at cell20.sc:1
24/09/06 14:53:45 INFO FileSourceScanExec: Planning scan with bin packing, max size: 4194304 bytes, open cost is considered as scanning 4194304 bytes.
24/09/06 14:53:45 INFO DAGScheduler: Registering RDD 61 (show at cell20.sc:1) as input to shuffle 2
24/09/06 14:53:45 INFO DAGScheduler: Got map stage job 14 (show at cell20.sc:1) with 1 output parti

+--------+
|count(1)|
+--------+
|     507|
+--------+



In [21]:
spark.sql("""
select count(*) from (
  select * from customer
  union all
  select * from customer
)""").show

24/09/06 14:53:48 INFO FileSourceStrategy: Pushed Filters: 
24/09/06 14:53:48 INFO FileSourceStrategy: Post-Scan Filters: 
24/09/06 14:53:48 INFO FileSourceStrategy: Pushed Filters: 
24/09/06 14:53:48 INFO FileSourceStrategy: Post-Scan Filters: 
24/09/06 14:53:48 INFO MemoryStore: Block broadcast_29 stored as values in memory (estimated size 198.4 KiB, free 4.5 GiB)
24/09/06 14:53:48 INFO MemoryStore: Block broadcast_29_piece0 stored as bytes in memory (estimated size 34.0 KiB, free 4.5 GiB)
24/09/06 14:53:48 INFO BlockManagerInfo: Added broadcast_29_piece0 in memory on ubuntu:34403 (size: 34.0 KiB, free: 4.5 GiB)
24/09/06 14:53:48 INFO SparkContext: Created broadcast 29 from show at cell21.sc:1
24/09/06 14:53:48 INFO FileSourceScanExec: Planning scan with bin packing, max size: 4194304 bytes, open cost is considered as scanning 4194304 bytes.
24/09/06 14:53:48 INFO MemoryStore: Block broadcast_30 stored as values in memory (estimated size 198.4 KiB, free 4.5 GiB)
24/09/06 14:53:48 INF

+--------+
|count(1)|
+--------+
|    1014|
+--------+



In [22]:
val retailDF = spark.read.format("json").load("data/retail_data.json")

24/09/06 14:54:02 INFO InMemoryFileIndex: It took 1 ms to list leaf files for 1 paths.
24/09/06 14:54:02 INFO InMemoryFileIndex: It took 0 ms to list leaf files for 1 paths.
24/09/06 14:54:02 INFO FileSourceStrategy: Pushed Filters: 
24/09/06 14:54:02 INFO FileSourceStrategy: Post-Scan Filters: 
24/09/06 14:54:02 INFO MemoryStore: Block broadcast_33 stored as values in memory (estimated size 198.5 KiB, free 4.5 GiB)
24/09/06 14:54:02 INFO MemoryStore: Block broadcast_33_piece0 stored as bytes in memory (estimated size 34.0 KiB, free 4.5 GiB)
24/09/06 14:54:02 INFO BlockManagerInfo: Added broadcast_33_piece0 in memory on ubuntu:34403 (size: 34.0 KiB, free: 4.5 GiB)
24/09/06 14:54:02 INFO SparkContext: Created broadcast 33 from load at cell22.sc:1
24/09/06 14:54:02 INFO FileSourceScanExec: Planning scan with bin packing, max size: 4194304 bytes, open cost is considered as scanning 4194304 bytes.
24/09/06 14:54:02 INFO SparkContext: Starting job: load at cell22.sc:1
24/09/06 14:54:02 INFO

retailDF: DataFrame = [CustomerID: string, Description: string ... 5 more fields]

In [23]:
retailDF.createOrReplaceTempView("retail")

In [26]:
val jDF = spark.sql("""
select c.*, r.*
from customer c
join retail r
on c.CustomerID == r.CustomerID
""")

jDF: DataFrame = [Address: string, Birthdate: string ... 12 more fields]

In [27]:
jDF.printSchema

root
 |-- Address: string (nullable = true)
 |-- Birthdate: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- CustomerID: string (nullable = true)
 |-- Email: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Username: string (nullable = true)
 |-- CustomerID: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- InvoiceNo: string (nullable = true)
 |-- Quantity: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- UnitPrice: string (nullable = true)



In [28]:
jDF.show(5, false)

24/09/06 14:56:39 INFO FileSourceStrategy: Pushed Filters: IsNotNull(CustomerID)
24/09/06 14:56:39 INFO FileSourceStrategy: Post-Scan Filters: isnotnull(CustomerID#11)
24/09/06 14:56:39 INFO FileSourceStrategy: Pushed Filters: IsNotNull(CustomerID)
24/09/06 14:56:39 INFO FileSourceStrategy: Post-Scan Filters: isnotnull(CustomerID#363)
24/09/06 14:56:39 INFO MemoryStore: Block broadcast_40 stored as values in memory (estimated size 198.4 KiB, free 4.5 GiB)
24/09/06 14:56:39 INFO MemoryStore: Block broadcast_40_piece0 stored as bytes in memory (estimated size 34.0 KiB, free 4.5 GiB)
24/09/06 14:56:39 INFO BlockManagerInfo: Added broadcast_40_piece0 in memory on ubuntu:34403 (size: 34.0 KiB, free: 4.5 GiB)
24/09/06 14:56:39 INFO SparkContext: Created broadcast 40 from $anonfun$withThreadLocalCaptured$1 at FutureTask.java:264
24/09/06 14:56:39 INFO FileSourceScanExec: Planning scan with bin packing, max size: 4194304 bytes, open cost is considered as scanning 4194304 bytes.
24/09/06 14:56:

+------------------------------------------------------+-------------------+--------------+----------+------------------------+---------------+----------------+----------+---------------------------------+---------------+---------+--------+---------+---------+
|Address                                               |Birthdate          |Country       |CustomerID|Email                   |Name           |Username        |CustomerID|Description                      |InvoiceDate    |InvoiceNo|Quantity|StockCode|UnitPrice|
+------------------------------------------------------+-------------------+--------------+----------+------------------------+---------------+----------------+----------+---------------------------------+---------------+---------+--------+---------+---------+
|Unit 1047 Box 4089\nDPO AA 57348                      |1994-02-20 00:46:27|United Kingdom|12346     |cooperalexis@hotmail.com|Lindsay Cowan  |valenciajennifer|12346     |MEDIUM CERAMIC TOP STORAGE JAR   |1/18/2011 10

In [29]:
jDF.select("CustomerID", "Name", "Birthdate", "InvoiceDate").show(5, false)

org.apache.spark.sql.AnalysisException: [AMBIGUOUS_REFERENCE] Reference `CustomerID` is ambiguous, could be: [`c`.`CustomerID`, `r`.`CustomerID`].

In [30]:
val jDF2 = spark.sql("""
select c.CustomerID, c.Name, c.Birthdate, r.InvoiceDate
from customer c
join retail r
on c.CustomerID == r.CustomerID
""")

jDF2: DataFrame = [CustomerID: string, Name: string ... 2 more fields]

In [31]:
jDF2.show(5, false)

24/09/06 14:59:34 INFO FileSourceStrategy: Pushed Filters: IsNotNull(CustomerID)
24/09/06 14:59:34 INFO FileSourceStrategy: Post-Scan Filters: isnotnull(CustomerID#11)
24/09/06 14:59:34 INFO FileSourceStrategy: Pushed Filters: IsNotNull(CustomerID)
24/09/06 14:59:34 INFO FileSourceStrategy: Post-Scan Filters: isnotnull(CustomerID#363)
24/09/06 14:59:34 INFO CodeGenerator: Code generated in 5.679604 ms
24/09/06 14:59:34 INFO MemoryStore: Block broadcast_45 stored as values in memory (estimated size 198.4 KiB, free 4.5 GiB)
24/09/06 14:59:34 INFO MemoryStore: Block broadcast_45_piece0 stored as bytes in memory (estimated size 34.0 KiB, free 4.5 GiB)
24/09/06 14:59:34 INFO BlockManagerInfo: Added broadcast_45_piece0 in memory on ubuntu:34403 (size: 34.0 KiB, free: 4.5 GiB)
24/09/06 14:59:34 INFO SparkContext: Created broadcast 45 from $anonfun$withThreadLocalCaptured$1 at FutureTask.java:264
24/09/06 14:59:34 INFO FileSourceScanExec: Planning scan with bin packing, max size: 4194304 bytes

+----------+---------------+-------------------+---------------+
|CustomerID|Name           |Birthdate          |InvoiceDate    |
+----------+---------------+-------------------+---------------+
|12346     |Lindsay Cowan  |1994-02-20 00:46:27|1/18/2011 10:01|
|12346     |Lindsay Cowan  |1994-02-20 00:46:27|1/18/2011 10:17|
|12347     |Katherine David|1988-06-21 00:15:34|12/7/2010 14:57|
|12347     |Katherine David|1988-06-21 00:15:34|12/7/2010 14:57|
|12347     |Katherine David|1988-06-21 00:15:34|12/7/2010 14:57|
+----------+---------------+-------------------+---------------+
only showing top 5 rows



In [32]:
spark.stop()

24/09/06 14:59:49 INFO SparkContext: SparkContext is stopping with exitCode 0.
24/09/06 14:59:49 INFO SparkUI: Stopped Spark web UI at http://ubuntu:4040
24/09/06 14:59:49 INFO MapOutputTrackerMasterEndpoint: MapOutputTrackerMasterEndpoint stopped!
24/09/06 14:59:49 INFO MemoryStore: MemoryStore cleared
24/09/06 14:59:49 INFO BlockManager: BlockManager stopped
24/09/06 14:59:49 INFO BlockManagerMaster: BlockManagerMaster stopped
24/09/06 14:59:49 INFO OutputCommitCoordinator$OutputCommitCoordinatorEndpoint: OutputCommitCoordinator stopped!
24/09/06 14:59:49 INFO SparkContext: Successfully stopped SparkContext
